# Cuaderno 13: Problemas de caminos más cortos 
# (shortest path problems)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido $D=(V,A)$; 
* un vector de costos $c \in \ZZ^{A}$  asociados a los arcos de $D$; 
* un nodo de salida $r$; y,
* un nodo de llegada $s$.

El *problema del camino más corto (shortest path problem, SPP)* consiste en encontrar un camino desde $r$ hasta $s$ cuyo costo sea mínimo. El costo de un camino se calcula sumando de los costos de sus arcos.

Suponer que $D$ no contiene circuitos de costo negativo. Utilizando variables binarias $x_{ij}$ para indicar si los arcos de $A$ son seleccionados o no para formar parte del camino, este problema puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(r, i) \in A} x_{ri} = 1,\\
&\sum_{(i, s) \in A} x_{is} = 1,\\
    &\sum_{(i, j) \in A} x_{ij} \leq 1, \quad \forall i \in V,\\
& \sum_{(j, i) \in A} x_{ji} - \sum_{(i, j) \in A} x_{ij} = 0, \quad \forall i \in V \setminus \tabulatedset{r,s},\\
& x_{ij} \in \tabulatedset{0, 1}, \quad \forall (i, j) \in A.
\end{align*}

La función objetivo mide la suma de los costos de los arcos seleccionados.

La primera restricción especifica que debe seleccionarse exactamente un arco saliente del nodo $r$. 

De manera similar, la segunda restricción indica que debe seleccionarse exactamente un arco entrante a $s$.

La tercera familia de restricciones determina que de cualquier nodo del grafo puede seleccionarse máximo un arco saliente.

Por último, la cuarta familia de restricciones requiere que la cantidad de arcos entrantes a un nodo $i \in V \setminus \tabulatedset{r,s}$ que sean seleccionados dentro de la solución debe coincidir con la cantidad de arcos salientes del mismo nodo.

Notar que todo camino de $r$ a $s$ satisface las cuatro restricciones del modelo. Por otra parte, existen otras clases de grafos que satisfacen estas restricciones. (Pensar en algún ejemplo.) Sin embargo, puede demostrarse que si $D$ no contiene *circuitos de costo negativo* entonces la solución óptima del modelo corresponde al camino de costo mínimo de $r$ a $s$.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los conjuntos y parámetros del modelo:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Nodos del grafo
V = gp.tuplelist(range(1,7))

# Arcos y sus costos 
A, c = gp.multidict({
  (1, 2):  3,
  (1, 3):  20,
  (2, 3):  10,
  (2, 4):  5,
  (4, 3):  2,
  (3, 5):  2,
  (4, 5):  5,
  (4, 6):  5,
  (5, 6):  2})

# Nodo de salida
r = 1

# Nodo de llegada
s = 6

# --- los valores a partir de aquí se calculan automáticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = gp.tuplelist([i for i in V if i!=r and i!=s])

La siguiente celda utiliza los módulos `NetworkX` y `matplotlib` para graficar esta instancia:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
D = nx.DiGraph()
D.add_nodes_from(V)
node_labels= {i : str(i) for i in V}
D.add_edges_from(A)
edge_labels = {(i,j) : str(c[i,j]) for (i,j) in A}
plt.figure(figsize=(10,5))
pos = {1 : (1,2), 2 : (2,3), 3 : (2,1), 4 : (3,3), 5 : (3,1), 6:(4,2)}
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=500)
nx.draw_networkx_edge_labels(D, pos, edge_labels)
plt.show()


Creamos ahora el objeto modelo y las variables de binarias de selección de arcos $x_{ij}$. Observar que las variables están indexadas por los conjuntos de arcos.

In [ ]:
# Crear el objeto modelo
m = gp.Model('shortest-path')

# Crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

Definimos la función objetivo a minimizar:

In [ ]:
m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

Definimos las restricciones del modelo:

In [ ]:
# Seleccionar un arco saliente de r
m.addConstr(x.sum(r,'*')  == 1, "grado_r")

# Seleccionar un arco entrante a s
m.addConstr(x.sum('*', s)  == 1, "grado_s")

# En cada nodo se selecciona máximo un arco saliente
m.addConstrs(
    (x.sum(i,'*') <= 1 for i in V), "grado_saliente")

# Balance de grados en los demás nodos
m.addConstrs(
    (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "balance")

Finalmente, resolvemos el modelo y mostramos la solución. Notar que los valores de las variables pueden recuperarse también empleando el método `getAttr`:

In [ ]:
# Resolver el modelo
m.optimize()

# Escribir la solución
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('Arcos seleccionados:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))

En la siguiente celda graficamos la solución empleando `networkx` y `matplotlib`:

In [ ]:
D = nx.DiGraph()
D.add_nodes_from(V)
node_labels= {i : str(i) for i in V}
D.add_edges_from(A)
edge_labels = {(i,j) : str(c[i,j]) for (i,j) in A}
plt.figure(figsize=(10,5))
pos = {1 : (1,2), 2 : (2,3), 3 : (2,1), 4 : (3,3), 5 : (3,1), 6:(4,2)}
edge_colors = ['#ff007f' if vx[i,j]>=0.1 else '#9dbaea' for (i,j) in list(D.edges())]
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=500, edge_color= edge_colors)
nx.draw_networkx_edge_labels(D, pos, edge_labels)
plt.show()

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos de programación lineal entera
# Problema de caminos más cortos (shortest path problem, SPP)
# Luis M. Torres (EPN 2025)

import random as rd
import gurobipy as gp
from gurobipy import GRB

# Número de nodos del grafo
n = 100

# Probabilidad de arcos
p = 0.1

# Valores (enteros) mínimos y máximo de costos para cada arco 
c_min, c_max = 1, 10

# --- los valores a partir de aquí se calculan automáticamente ---

# Conjunto de nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Agregar el camino P0 = (1, 2, 3,..., n) para tener al menos una solución factible
A = [(i,i+1) for i in V if i!=n]

# Agregar los demás arcos con probabilidad p
A+= [(i, j) for i in V for j in V if j not in [i, i+1] and rd.random()<=p]

# Definir costos aleatorios sobre los arcos, en el rango {c_min,...,c_max}
c = {(i,j) : rd.randint(c_min, c_max) for (i,j) in A}

# Fijar costos máximos sobre P0
for i in V[:-1]:
    c[i,i+1] = c_max

# Nodo de salida
r = 1

# Nodo de llegada
s = n

# nodos internos: Vi := V \ {s, t}
Vi = gp.tuplelist([i for i in V if i!=r and i!=s])

try:
    # Crear el objeto modelo
    m = gp.Model('shortest-path')

    # Crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Definir la función objetivo
    m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

    # Seleccionar un arco saliente de r
    m.addConstr(x.sum(r,'*')  == 1, "grado_r")

    # Seleccionar un arco entrante a s
    m.addConstr(x.sum('*', s)  == 1, "grado_s")

    # En cada nodo se selecciona máximo un arco saliente
    m.addConstrs(
        (x.sum(i,'*') <= 1 for i in V), "grado_saliente")
    # Balance de grados en los demás nodos
    m.addConstrs(
        (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "balance")
    
    # Resolver el modelo
    m.optimize()

    # Escribir la solución
    if m.SolCount > 0:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('Arcos seleccionados:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} -> {}'.format(i, j))
            
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

In [ ]:
m.write('caminos.lp')